<a href="https://colab.research.google.com/github/lalanglarano/TAKLUBAN-FILIPINO-NATIVE-LANGUAGE-PROFANE-DETECTION/blob/old-main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

In [ ]:
dataset = load_dataset('csv', data_files={'train': '/content/DATASETS/train (1).csv', 'validation': '/content/DATASETS/val (1).csv'})

# Load pre-trained BERT tokenizer and model for sequence classification (2 output classes)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Tokenization function for dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', max_length=128, truncation=True)

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Set format for PyTorch tensors
tokenized_datasets.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=0.00001,  # 1e-5 as shown in your hyperparameters
    per_device_train_batch_size=64,  # Specified batch size
    per_device_eval_batch_size=64,
    num_train_epochs=3,  # Train for 3 epochs (adjust if needed)
    weight_decay=0.01,
    logging_dir='./logs',
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

# Train the model
trainer.train()

# Save the model
trainer.save_model('./trained_bert')

# Evaluate the model
results = trainer.evaluate()

# Print results
print(results)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Map:   0%|          | 0/10995 [00:00<?, ? examples/s]

Map:   0%|          | 0/1375 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.361791
2,No log,0.337416
3,0.412000,0.322195


{'eval_loss': 0.3221949338912964, 'eval_runtime': 9.9407, 'eval_samples_per_second': 138.32, 'eval_steps_per_second': 2.213, 'epoch': 3.0}


In [ ]:
!pip install safetensors

In [ ]:
from safetensors.torch import load_file

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load model configuration and weights
model = BertForSequenceClassification.from_pretrained(
    './trained_bert/',
    config='./trained_bert/config.json',
    from_tf=False
)

# Load the weights from safetensors format
state_dict = load_file("./trained_bert/model.safetensors")
model.load_state_dict(state_dict)

# Set the model to evaluation mode
model.eval()

# Example usage
sentence = "putangina mo inaway mo na naman kaibigan mo"
inputs = tokenizer(sentence, return_tensors="pt")

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Get predictions
if probabilities[0, 1] > 0.8:  # Profane if probability of class 1 is greater than 0.8
    print("Profane")
else:
    print("Not Profane")


probabilities = torch.softmax(logits, dim=-1)
print(f"Probabilities: {probabilities}")



Not Profane
Probabilities: tensor([[0.0521, 0.9479]])
